In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# put Partitive-Files data inside drive
%cd /content/drive/MyDrive/Partitive-Files/
!mkdir csv-data
!ls

/content/drive/MyDrive/Partitive-Files
mkdir: cannot create directory ‘csv-data’: File exists
arg_scorer.py			   partitive_group_nombank.clean.test
csv-data			   partitive_group_nombank.clean.train
%_nombank.clean.dev		   __pycache__
%_nombank.clean.test		   README-percent-partitive.txt
%_nombank.clean.train		   results.txt
old_partitive.txt		   sstcls_1.dat
partitive_group_nombank.clean.dev  test-trainer


In [ ]:
!pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 26.3 MB/s 
     |████████████████████████████████| 452 kB 55.0 MB/s 
     |████████████████████████████████| 7.6 MB 41.8 MB/s 
     |████████████████████████████████| 182 kB 68.5 MB/s 
     |████████████████████████████████| 212 kB 75.4 MB/s 
     |████████████████████████████████| 132 kB 61.6 MB/s 
     |████████████████████████████████| 127 kB 71.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
from transformers import BertConfig
config = BertConfig.from_pretrained(
    "bert-base-uncased"
) 

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
config

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

## data preprocessing

In [ ]:
import csv  
from datasets import load_dataset

In [ ]:
# load data
# assumptions: every sentence should contain just one ARG1
def preprocess_data(file_path):
    sentences = []
    cur_sentence = []
    arg1_tags = []
    cur_tag = []

    with open(file_path, 'r') as f:
        for line in f.readlines():
            if line != '\n':
                line_list = line.split() # ['insiders', 'NNS', 'I-NP', '6', '0', 'ARG1']
                cur_sentence.append(line_list[0]) # extract word
                cur_tag.append(1) if line_list[-1] == 'ARG1' else cur_tag.append(0)
            else: # if new line: appends words from previous sentence to sentences
                sentences.append(' '.join(cur_sentence).replace('COMMA', ','))
                cur_sentence = []

                arg1_tags.append(cur_tag)
                cur_tag = []
    
    tokens = [s.split() for s in sentences]
    labels = []
    for tag in arg1_tags:
        if 1 in tag:
            labels.append(tag.index(1))
        else:
            labels.append(0)
    return zip(range(len(sentences)), sentences, tokens, arg1_tags, labels)


train_data = preprocess_data('%_nombank.clean.train')
dev_data = preprocess_data('%_nombank.clean.dev')
test_data = preprocess_data('%_nombank.clean.test')

In [ ]:
# store data in csv files
def write_to_csv(data, ouput_dir):
    with open(ouput_dir, 'w', encoding='UTF8') as f:
        header = ['id', 'texts', 'tokens', 'arg1_tags', 'labels']
        writer = csv.writer(f)
        writer.writerow(header)
        writer.writerows(data)

write_to_csv(train_data, './csv-data/train.csv')
write_to_csv(dev_data, './csv-data/dev.csv')
write_to_csv(test_data, './csv-data/test.csv')

In [ ]:
# load dataset from csv
data_files = {"train": "train.csv", "dev": "dev.csv", "test": "test.csv"}
dataset = load_dataset('./csv-data', data_files=data_files)

Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #2:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating dev split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/csv-data-500d841df67ed688/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'texts', 'tokens', 'arg1_tags', 'labels'],
        num_rows: 2174
    })
    dev: Dataset({
        features: ['id', 'texts', 'tokens', 'arg1_tags', 'labels'],
        num_rows: 83
    })
    test: Dataset({
        features: ['id', 'texts', 'tokens', 'arg1_tags', 'labels'],
        num_rows: 150
    })
})

In [ ]:
dataset["train"][0]

{'id': 0,
 'texts': 'But about 25 % of the insiders , according to SEC figures , file their reports late .',
 'tokens': "['But', 'about', '25', '%', 'of', 'the', 'insiders', ',', 'according', 'to', 'SEC', 'figures', ',', 'file', 'their', 'reports', 'late', '.']",
 'arg1_tags': '[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]',
 'labels': 6}

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def tokenize_function(example):
    return tokenizer(example["texts"], truncation=True)

In [ ]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
from transformers import TrainingArguments, AutoModelForSequenceClassification, DataCollatorWithPadding

training_args = TrainingArguments("test-trainer")
training_args.num_train_epochs = 10
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
checkpoint = 'bert-base-uncased'
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=52).to("cuda")
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["dev"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

NameError: ignored

In [ ]:
training_args

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tokens, texts, id, arg1_tags. If tokens, texts, id, arg1_tags are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2174
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2720
  Number of trainable parameters = 109522228
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the

Step,Training Loss
500,2.952500
1000,2.168100
1500,1.782300
2000,1.451400
2500,1.246100


Saving model checkpoint to test-trainer/checkpoint-500
Configuration saved in test-trainer/checkpoint-500/config.json
Model weights saved in test-trainer/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-500/special_tokens_map.json
Saving model checkpoint to test-trainer/checkpoint-1000
Configuration saved in test-trainer/checkpoint-1000/config.json
Model weights saved in test-trainer/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-1000/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to test-trainer/checkpoint-1500
Configuration saved in test-trainer/checkpoint-1500/config.json
Model weights saved in test-trainer/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-1500/tokenizer_config.json
Special tokens file saved

TrainOutput(global_step=2720, training_loss=1.855376782136805, metrics={'train_runtime': 456.288, 'train_samples_per_second': 47.645, 'train_steps_per_second': 5.961, 'total_flos': 608077899686880.0, 'train_loss': 1.855376782136805, 'epoch': 10.0})

In [ ]:
predictions = trainer.predict(tokenized_datasets["dev"])

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: arg1_tags, id, texts, tokens. If arg1_tags, id, texts, tokens are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 83
  Batch size = 8


In [ ]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 1.7 MB/s 


In [ ]:
import evaluate
import numpy as np

metric = evaluate.load("glue", "mnli")
preds = np.argmax(predictions.predictions, axis=-1)
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.4819277108433735}

In [ ]:
preds

array([11,  9, 23,  4,  4,  8, 12, 12,  9,  2,  4,  6,  3, 24, 24, 20,  2,
       13, 20, 15,  2, 10,  4, 37, 20, 29, 12, 12, 20, 20, 21, 11, 11,  4,
       27, 26,  5,  0,  3,  4, 15, 11,  8,  0, 14, 14, 14,  0,  0,  6,  6,
       50,  9,  2,  1,  0,  0, 10, 35, 12,  1,  1,  5,  1,  5,  5,  1,  2,
        2,  1,  2,  4, 10,  1,  8,  8, 20, 15,  0, 15, 10, 11,  0])

In [ ]:
predictions.label_ids

array([11, 10, 23, 11,  4,  0, 20, 11,  9,  2,  4,  5,  3,  7,  7, 23,  2,
       16, 21, 14,  2,  9,  4, 38, 23, 38, 12, 12, 16, 28, 22, 10, 10,  4,
       23, 24,  6,  0,  3,  4, 15,  9,  8,  0, 21, 30, 12,  0,  0,  4,  4,
       49,  4,  2,  1,  0,  0,  1, 35, 12, 12,  1,  5,  1,  4, 18,  1, 21,
        2,  1,  2,  3, 10,  1,  8, 19, 21, 17,  0, 17, 10,  8,  0])

In [ ]:
## test
test_logits = trainer.predict(tokenized_datasets["test"])
test_preds = np.argmax(test_logits.predictions, axis=-1)
print(test_preds)

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: arg1_tags, id, texts, tokens. If arg1_tags, id, texts, tokens are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 150
  Batch size = 8


[26 19  1 15  3  3  2  9 15 17 17 18 15 10 15 15 15 26 11 22 13 22 18 13
 12 27 15  4  4 25 26 29  6  7  7  4  0 29 29 37 29 29 29 18 11 13 15  0
  0 18 18 18  5  6  6  7  0  8  1  5 12 15 13 13 11  9  7 27 11 18  6  6
  2  2  6 12  6  6  4  6 35 10 14 14  2  2  0  0  1  1  9  0  0  0  0  1
  0  0  0 27  7  5 19 29 29 27  3  1  8  8  9  8 20 18  6  4  8 12 12  2
  7  3 31 13  3  5  3  3  0  2  6  6  8  8  0 18 24 15 15 19 13  0  3  3
 11 26 18  8  8 31]


In [ ]:
results = []
sentence_idx = 0
word_idx = 0
with open('%_nombank.clean.test', 'r') as f:
    for line in f.readlines():
        if line == '\n':
            results.append('\n')
            sentence_idx += 1
            word_idx = 0
        else:
            if word_idx == test_preds[sentence_idx]:
                results.append(line.split()[0] + ' ARG1 \n')
            else:
                results.append(line.split()[0] + '\n')
            word_idx += 1

with open('results.txt', 'w') as f:
    f.write(''.join(results))

In [ ]:
from arg_scorer import score_file_with_NNP_adjustment

In [ ]:
score_file_with_NNP_adjustment('%_nombank.clean.test', 'results.txt', 'ARG1')

System [26, 62, 78, 102, 117, 154, 169, 194, 228, 259, 290, 322, 353, 380, 415, 439, 463, 498, 522, 556, 581, 613, 638, 661, 684, 724, 742, 756, 784, 833, 868, 902, 913, 931, 955, 976, 1013, 1057, 1094, 1136, 1174, 1219, 1264, 1298, 1312, 1329, 1346, 1352, 1384, 1453, 1478, 1490, 1524, 1569, 1615, 1642, 1664, 1675, 1693, 1739, 1781, 1799, 1828, 1864, 1891, 1948, 1972, 2006, 2023, 2063, 2099, 2128, 2161, 2222, 2254, 2279, 2302, 2333, 2393, 2414, 2456, 2485, 2502, 2527, 2550, 2573, 2597, 2611, 2633, 2645, 2674, 2703, 2732, 2762, 2780, 2809, 2838, 2894, 2913, 2959, 2988, 3040, 3084, 3120, 3136, 3165, 3235, 3263, 3292, 3322, 3355, 3380, 3403, 3427, 3457, 3480, 3513, 3532, 3566, 3590, 3643, 3663, 3674, 3705, 3734, 3762, 3780, 3795, 3815, 3839, 3865, 3898, 3923, 3973, 4024, 4053, 4072, 4094, 4129, 4160, 4206, 4237, 4276, 4307, 4329, 4341, 4373, 4424]
Answers [9, 62, 78, 103, 116, 154, 169, 195, 229, 257, 288, 320, 355, 380, 421, 438, 462, 501, 523, 557, 581, 612, 637, 660, 686, 722, 742, 756